In [1]:
## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
file_to_load = "../Weather_Database/WeatherPy_Database.csv"
city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Komsomolskiy,UZ,40.43,71.72,71.60,28,74,4.70,broken clouds
1,1,Kuryk,KZ,43.20,51.65,84.49,17,26,15.12,scattered clouds
2,2,Rikitea,PF,-23.12,-134.97,72.77,82,4,15.97,clear sky
3,3,Ushuaia,AR,-54.80,-68.30,42.80,75,40,19.46,scattered clouds
4,4,Kapaa,US,22.08,-159.32,78.80,74,75,13.87,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Kuryk,KZ,43.20,51.65,84.49,17,26,15.12,scattered clouds
4,4,Kapaa,US,22.08,-159.32,78.80,74,75,13.87,broken clouds
5,5,Castro,BR,-24.79,-50.01,82.33,32,1,10.51,clear sky
6,6,Laredo,US,27.51,-99.51,75.20,94,20,4.70,few clouds
11,11,Maltahohe,NaN,-24.83,16.98,83.84,14,0,15.21,clear sky
13,13,Kuching,MY,1.55,110.33,75.20,100,40,2.57,scattered clouds
14,14,Guerrero Negro,MX,27.98,-114.06,87.17,46,0,5.12,clear sky
17,17,Saint George,US,37.10,-113.58,82.99,14,1,3.36,clear sky
27,27,Lorengau,PG,-2.02,147.27,81.75,81,8,10.74,light rain
29,29,Hambantota,LK,6.12,81.12,75.20,100,90,4.70,overcast clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0     267
City           267
Country        266
Lat            267
Lng            267
Max Temp       267
Humidity       267
Cloudiness     267
Wind Speed     267
Description    267
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_city_df = preferred_cities_df.dropna()
preferred_city_df.count()

Unnamed: 0     266
City           266
Country        266
Lat            266
Lng            266
Max Temp       266
Humidity       266
Cloudiness     266
Wind Speed     266
Description    266
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_city_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()
hotel_df.head()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Kuryk,KZ,84.49,scattered clouds,43.20,51.65,
4,Kapaa,US,78.80,broken clouds,22.08,-159.32,
5,Castro,BR,82.33,clear sky,-24.79,-50.01,
6,Laredo,US,75.20,few clouds,27.51,-99.51,
13,Kuching,MY,75.20,scattered clouds,1.55,110.33,
14,Guerrero Negro,MX,87.17,clear sky,27.98,-114.06,
17,Saint George,US,82.99,clear sky,37.10,-113.58,
27,Lorengau,PG,81.75,light rain,-2.02,147.27,
29,Hambantota,LK,75.20,overcast clouds,6.12,81.12,
33,Le Vauclin,MQ,89.60,few clouds,14.55,-60.84,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row['Lng']
    params['location']= f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except(IndexError):
        print("Hotel not found.. skipping")
    

Hotel not found.. skipping
Hotel not found.. skipping
Hotel not found.. skipping
Hotel not found.. skipping
Hotel not found.. skipping
Hotel not found.. skipping
Hotel not found.. skipping
Hotel not found.. skipping
Hotel not found.. skipping
Hotel not found.. skipping
Hotel not found.. skipping
Hotel not found.. skipping


In [20]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Kuryk,KZ,84.49,scattered clouds,43.20,51.65,Gostinitsa Kuryk - Otel' Dlya Vsekh Kategoriy ...
4,Kapaa,US,78.80,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
5,Castro,BR,82.33,clear sky,-24.79,-50.01,CHACARA BAILLY
6,Laredo,US,75.20,few clouds,27.51,-99.51,La Quinta Inn by Wyndham Laredo I-35
13,Kuching,MY,75.20,scattered clouds,1.55,110.33,Hilton Kuching
...,...,...,...,...,...,...,...
714,Butwal,NP,78.89,light rain,27.70,83.45,Hotel Sindoor Pvt. Ltd.
715,Miramar,US,90.00,few clouds,25.99,-80.23,Stadium Hotel
716,Pinar Del Rio,CU,86.68,clear sky,22.42,-83.70,Hostal La Colina Dunay y Raul
717,Mahibadhoo,MV,82.85,overcast clouds,3.78,72.97,Omadhoo Inn


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City           266
Country        266
Max Temp       266
Description    266
Lat            266
Lng            266
Hotel Name     266
dtype: int64

In [24]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))